# Fact Extraction and VERification task

In [1]:
import os
import numpy as np
import pandas as pd
import json
import string
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from keras_preprocessing import sequence, text
import wikipedia

In [2]:
#If the file is downloaded to the format .json1 you have to rename the file and remove the "1" at the end.
#Otherwise it's impossible to read the file
datafile_fever = 'data/train.json'
datafile_shared_task = 'data/shared_task_dev.json'
wiki_pages_folder = 'data/wiki-pages'

In [3]:
df_train = pd.read_json(datafile_fever, lines=True)#,orient='table')
df_train.shape

(145449, 5)

In [4]:
df_task = pd.read_json(datafile_shared_task, lines=True)
df_task.shape

(19998, 5)

## Data exploration

In [5]:
df_train['label'].value_counts()

SUPPORTS           80035
NOT ENOUGH INFO    35639
REFUTES            29775
Name: label, dtype: int64

In [7]:
35000/(80035+35639+29775)

0.24063417417789054

In [14]:
df_train.head()

,id,verifiable,label,claim,evidence
0,75397,VERIFIABLE,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,"[[[92206, 104971, Nikolaj_Coster-Waldau, 7], [..."
1,150448,VERIFIABLE,SUPPORTS,Roman Atwood is a content creator.,"[[[174271, 187498, Roman_Atwood, 1]], [[174271..."
2,214861,VERIFIABLE,SUPPORTS,"History of art includes architecture, dance, s...","[[[255136, 254645, History_of_art, 2]]]"
3,156709,VERIFIABLE,REFUTES,Adrienne Bailon is an accountant.,"[[[180804, 193183, Adrienne_Bailon, 0]]]"
4,83235,NOT VERIFIABLE,NOT ENOUGH INFO,System of a Down briefly disbanded in limbo.,"[[[100277, None, None, None]]]"


In [59]:
def find_possibleFiles(firstLetter):
    if firstLetter == '0' or firstLetter =='1' or firstLetter =='2' or firstLetter =='3' or firstLetter =='4' or firstLetter =='5' or firstLetter =='6' or firstLetter =='7' or firstLetter =='8' or firstLetter =='9':
        return [1,2,3,4,5,6,7]
    elif firstLetter == 'A':
        return [5,6,7,8,9,10,11,12,13]
    elif firstLetter == 'B':
        return [11,12,13,14,15,16,17,18,19,20]
    elif firstLetter == 'C':
        return [18,19,20,21,22,23,24,25,26]
    elif firstLetter == 'D':
        return [24,25,26,27,28,29,30,31]
    elif firstLetter == 'E':
        return [29,30,31,32,33,34,35]
    elif firstLetter == 'F':
        return [33,34,35,36,37]
    elif firstLetter == 'G':
        return [35,36,37,38,39,40,41,42]
    elif firstLetter == 'H':
        return [39,40,41,42,43,44,45,46]
    elif firstLetter == 'I':
        return [44,45,46,47,48]
    elif firstLetter == 'J':
        return [46,47,48,49,50,51,52]
    elif firstLetter == 'K':
        return [50,51,52,53,54,55,56,57]
    elif firstLetter == 'L':
        return [55,56,57,58,59,60,61,62,63]
    elif firstLetter == 'M':
        return [61,62,63,64,65,66,67,68,69,70]
    elif firstLetter == 'N':
        return [68,69,70,71,72,73,74]
    elif firstLetter == 'O':
        return [72,73,74,75,76]
    elif firstLetter == 'P':
        return [74,75,76,77,78,79,80,81]
    elif firstLetter == 'Q':
        return [79,80,81]
    elif firstLetter == 'R':
        return [79,80,81,82,83,84,85]
    elif firstLetter == 'S':
        return [85,86,87,88,89,90,91,92,93,94,95,96]
    elif firstLetter == 'T':
        return [94,95,96,97,98,99,100,101,102]
    elif firstLetter == 'U':
        return [100,101,102]
    elif firstLetter == 'V':
        return [100,101,102,103,104]
    elif firstLetter == 'W':
        return [102,103,104,105,106,107,108]
    elif firstLetter == 'X':
        return [107,108]
    elif firstLetter == 'Y':
        return [107,108]
    elif firstLetter == 'Z':
        return [107,108]

In [61]:
def train_findEvidence():
    sub = pd.DataFrame()
    for nb_rows in tqdm(range(len(df_train['evidence'][0:5]))):
        row_over = False
        while row_over != True:
            if df_train['verifiable'][nb_rows] == 'NOT VERIFIABLE':
                row_over = True
            else:
                for nb_evidence in range(len(df_train['evidence'][nb_rows])):
                    for nb_sentences in range(len(df_train['evidence'][nb_rows][nb_evidence])):
                        #print(df_train['evidence'][nb_rows][nb_evidence][nb_sentences][2][0])
                        possible_files = find_possibleFiles(df_train['evidence'][nb_rows][nb_evidence][nb_sentences][2][0])
                        #print(possible_files)
                    for nb_files in range(len(possible_files)):
                        if possible_files[nb_files]<10 and possible_files[nb_files]>=1:
                            article = "00"+str(possible_files[nb_files])
                        elif possible_files[nb_files]>9 and possible_files[nb_files]<100:
                            article = "0"+str(possible_files[nb_files])
                        else:
                            article = str(possible_files[nb_files])
                        wiki_file = pd.read_json(wiki_pages_folder+'/wiki-'+article+'.json', lines=True)
                        for articles in range(len(wiki_file['id'])):
                            if wiki_file['id'][articles]==df_train['evidence'][nb_rows][nb_evidence][nb_sentences][2]:
                                #print(wiki_file['id'][articles])
                                
                                break
                row_over = True
    return 0

In [62]:
train_findEvidence()


  0%|          | 0/5 [00:00<?, ?it/s]

Fox_Broadcasting_Company



 20%|██        | 1/5 [00:14<00:58, 14.74s/it]

Roman_Atwood
Roman_Atwood



 40%|████      | 2/5 [00:52<01:05, 21.74s/it]

History_of_art



 60%|██████    | 3/5 [01:14<00:43, 21.77s/it]

Adrienne_Bailon



100%|██████████| 5/5 [01:39<00:00, 19.92s/it]


0

# Creation of a redeuced SNLI csv file

## Not able to retrieve sentences from FEVER, so use an other dataset

In [77]:
snli_train = 'data/snli/snli_train.json'
df_snli = pd.read_json(snli_train, lines=True)#,orient='table')
df_snli.shape

(550152, 10)

In [78]:
df_snli= df_snli.drop(['captionID', 'annotator_labels','pairID','sentence1_binary_parse','sentence1_parse','sentence2_binary_parse','sentence2_parse'], axis=1)

In [79]:
df_snli.head()

,gold_label,sentence1,sentence2
0,neutral,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.
1,contradiction,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette."
2,entailment,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse."
3,neutral,Children smiling and waving at camera,They are smiling at their parents
4,entailment,Children smiling and waving at camera,There are children present


In [82]:
df_snli.to_csv('data/train_snli.csv')

In [83]:
df_snli['gold_label'].value_counts()

entailment       183416
contradiction    183187
neutral          182764
-                   785
Name: gold_label, dtype: int64